# Infotheory Surprisal from Language Models Tutorial

In this tutorial we will reproduce a part of the experiments in [1] on the Natural Stories SPR (self-paced reading) corpus [2]. 
We will calculate surprisal values with different sizes of GPT2. The model (sizes) are provided by Hugging Face Transformers, and are pretrained on large-scale English text data.

### 1. Surprisal Toolkit Warmup

As a warm-up to the tutorial, let's familiarize with calculating surprisal over tokens using an interactive toolkit. We will revisit this toolkit later to quickly retrieve results.

First, visit the Surprisal Toolkit website: [our-university-server-URL-here].

The Surprisal Toolkit allows you to enter text or upload .txt files in order to calculate surprisal scores, using probabilities processed by several large language models available from Hugging Face (https://huggingface.co/models).

#### Step 1: Process a sentence or two in the text box.

- Choose any model and click `Compute Surprisal`. Note that the context for calculating surprisal is limited to each line of text.
- *If you choose a gpt2 model, you will need to also check "Prepend token".*

#### Step 2: View the results preview. 
- Scroll through the per-token results in the `Results` tab. 

- *What do you notice about the tokens compared to the words in the original sentence(s)? What do you notice about the surprisal values across tokens? Between tokens of the same word? At the end of a sentence? At the start of the next sentence?*

- Click `Download Results` to save the calculations.

#### Step 3: View plotted results.
- Visit the `Plots` tab to view a plot of tokens by surprisal, per line of text. Select a sentence to view its plot.

- Hover over the plot to view a row of icons. <img src="media/plot-icons-screenshot.png" width="200"> These allow you to navigate the plot by zooming in and out, and panning across the axes.

    - Zoom into a small section of the plotted sentence by clicking the `Zoom` icon and dragging a box over your preferred selection. 
    - Click `Pan` to move across the sentence.

- *How does surprisal change across tokens in the sentence? Which token transitions have the highest surprisal scores? Which transitions show a decrease in surprisal? Do the model estimates appear reasonable based on your own expectations?*
    
- Save the plot by clicking the `Download plot as a png` :camera: icon.

#### Step 4: Repeat and compare plots.
- Repeat the surprisal computation with the **same input** and a **different, larger or smaller model** from the list. Download the results.tsv file and the plot png. 
- What do you notice about the results between models?

#### Step further: exploration.
If you'd like to explore further:
- Notice the option to check `Prepend token`. This is relevant for the gpt2 models based on how they process text. What happens to the results when you leave it unchecked?
- How consistent are these models? Try entering the same sentence more than once into the textbox.
- Can you predict a very surprising or unsurprising sentence, grammatical or not, according to a given model? Find 2 sentences to compare and plot to illustrate your hypothesized prediction.


### 2. Install required packages

We assume that you are on a Linux machine, or use WSL if you are on Windows. 
The recommended way to run this tutorial is with a virtual Python environment on your local machine, but you can also use [colab](https://colab.research.google.com/), which already has most of the dependencies installed, and comes with optional GPU-acceleration.

If you are not on colab, you will have to install some dependencies first. In colab you only need to install the ```lmerTest``` and the Python packages other than PyTorch (see below).

- Install the appropriate PyTorch version from [here](https://pytorch.org/get-started/locally/)

- We will use the ```lmerTest``` package to fit a linear mixed effect model (LME) to predict self-paced reading times from the surprisal values derived from the two models.

In [ ]:
# %%capture
!R -e 'oo <- options(repos = "https://cran.r-project.org/"); install.packages("Matrix"); install.packages("lme4"); install.packages("lmerTest"); options(oo)'

- ```pymer4``` is a Python wrapper around the ```lmerTest``` package, and saves us from messing with R code in a notebook.
- ```transformers``` is the Python library we use to load and run the pretrained models.

In [ ]:
%pip install transformers pymer4 scikit-learn

Tip: if you are running Jupyter Notebook locally, and you find your computer short on memory, uncomment and try the following:

In [ ]:
# # Remove all variables from memory, except for those defined in a configuration file
# %reset -f

# # Collect and free memory no longer in use by the notebook
# import gc
# gc.collect()

### 3. Get Natural Stories SPR data

Let's download the corpus from Github:

In [ ]:
!git clone https://github.com/languageMIT/naturalstories.git

The raw data is in a file ```all_stories.tok```, with each word assigned to a story ('item') and position ('zone'):

In [ ]:
import os
natural_stories_path = os.path.join("naturalstories/naturalstories_RTS", "all_stories.tok")

Let's load the data in a DataFrame object:

In [ ]:
import pandas as pd

stories = []
df = pd.read_csv(natural_stories_path, sep='\t', header=0)

print(df.head())

Save all stories in a single text file with one story per line. These will be used to get surprisal scores for each word in the corpus, via the Surprisal Toolkit. Putting each story in one line tells the toolkit that it should maximize the model's context window, e.g. 1024 tokens for the smallest GPT2 model. The tokens of a story that don't fit into a context window will be placed at the beginning of a new context window, and therefore be conditioned on a smaller number of tokens.

In [ ]:
with open("naturalstories.txt", "w") as f:
    for item in df.item.unique():
        df_item = df[df.item == item]
        text = " ".join(df_item.word)
        f.write(text + ("\n" if item < 10 else ""))

Now, load the reading time data in `naturalstories/naturalstories_RTS/processed_RTS.tsv` into a Dataframe object

In [ ]:
subject_rts_path = os.path.join("naturalstories/naturalstories_RTS", "processed_RTs.tsv")
df_rts = pd.read_csv(subject_rts_path, sep="\t")
df_rts.head(1)

The reading time dataframe has several columns that are relevant for the reading time prediction task: `word` (the form of the word as it was presented to the participant of the reading time study), `item` (the id of the story), `RT` (the raw reading time in milliseconds) and `nItem` (the frequency if the word in the Natural Stories corpus). While `RT` is the response variable we are interested in, `item`, `word`, and `nItem` will be used as predictors.

`WorkerId` is the id of the participant. As we are not interested in *individual* variability, we average the `RT` column over all `WorkerIds`:

In [ ]:
df_rts = df_rts.drop(columns=["WorkerId", "WorkTimeInSeconds", "correct"]). \
    groupby(by=["item", "zone", "word", "nItem"]). \
    mean(). \
    reset_index()

# Drop word column
df_rts = df_rts.drop(["word"], axis=1)

#### 3.1 Surprisal from GPT2 via LM Toolkit

Instead of calculating surprisal by hand, we will instead use the functionality of the LM Toolkit to get surprisal on Natural Stories from 3 sizes of GPT2: `gpt2` (125m parameters, the small or base model, listed as `gpt2-small`), `gpt2-medium` (350m parameters) and `gpt2-large` (774m parameters). 

1. Upload the text file `naturalstories.txt` created above via the `Choose File` button, select `gpt2` as the model, check the `Prepend Token` box and calculate surprisal.
2. Download the `results.tsv` file and rename it to `results_gpt2_small.tsv`
3. Repeat for the other sizes of GPT2

The resulting file should look like this:

```
sentence_id	token	surprisal	token_id
0	If	7.7582244873046875	1532
0	Ġyou	0.8087347149848938	345
0	Ġwere	5.417769908905029	547
0	Ġto	2.095001459121704	284
0	Ġjourney	14.621973037719727	7002
0	Ġto	2.1376800537109375	284
0	Ġthe	2.6726841926574707	262
0	ĠNorth	6.569890975952148	2258
0	Ġof	7.013443946838379	286
0	ĠEngland	2.3848724365234375	4492
0	,	2.2026619911193848	11
0	Ġyou	1.6058685779571533	345
0	Ġwould	1.346909761428833	561
0	Ġcome	6.167995452880859	1282
```

The `sentence_id` column contains the id of the story, `token` the words or subtokens the story was split into by GPT2's tokenizer, and `surprisal` contains the per-token surprisal. Since we want to model reading times on the *word* level, we have to reconstruct the words in the input by merging their subtokens. Word-level surprisal is obtained by *summing* the surprisal of the subtokens belonging to a particular word, following the chain rule of conditional probabilities.

#### 3.2 Word-level surprisal from subtoken-level surprisal

Start by loading the stories scored with `gpt2` into a dataframe:

In [ ]:
df_scored = pd.read_csv("results_gpt2_small.tsv", sep="\t")
# df_scored = pd.read_csv("results_gpt2_medium.tsv", sep="\t")
# df_scored = pd.read_csv("results_gpt2_large.tsv", sep="\t")
df_scored.iloc[10:30]

We observe that most words are not split into subtokens. The affixed "Ġ" indicates that the token is the first token of an orthographic word, and subwords are indicated by the absence of the "Ġ". This can be seen with "Ġmo", "ors", which we have to merge into "moors" and whose surprisal values need to be summed: $15.3574 + 9.2472 = 24.6046$. This needs to be done for all words that have been split into subwords.

Finish the implementation of the function `get_word_surprisal` below, looping over all tokens in the results file, merging subtokens per word and summing their surprisals (you can test your code by running the code cell immediately below):

In [ ]:
from typing import List, Tuple

def get_word_surprisal(tokens: List[str], token_surprisal: List[float]) -> Tuple[List[str], List[float]]:
    """ Calculates per-word surprisal from a list of subword tokens and subword surprisal values
        by adding up the surprisal values of the subword tokens.
    Parameters
    ----------
    tokens : List[str]
        The list of tokens. Each word-initial token is expected to start with the character 'Ġ',
        which is added by the GPT2 tokenizer.
    token_surprisal : List[float]
        The list of surprisal values corresponding to the subword tokens.

    Returns
    -------
    Tuple[List[str], List[float]]
        The reassembled words and their surprisal values.
    """
    word_surprisal = []
    words = []

    # TODO: your code here

    return words, word_surprisal

In [ ]:
df_story_1 = df_scored[df_scored.sentence_id == 0]

words, word_surprisals = get_word_surprisal(df_story_1.token.tolist(), df_story_1.surprisal.tolist())

print(words)
print(word_surprisals)

Now we are ready to obtain word-level surprisal for all stories! The for loop below will add 6 more columns to the dataframe: 3 columns with surprisal from the current word `surp_0` and surprisal from the two preceding words (`surp_1`, `surp_2`), which will all be used as predictors for `word_0`'s reading time.

In [ ]:
story_surprisals = {}

for sentence_id in df_scored.sentence_id.unique():
    
    df_item_scored = df_scored[df_scored.sentence_id == sentence_id]

    words, word_surprisals = get_word_surprisal(df_item_scored.token.tolist(), df_item_scored.surprisal.tolist())

    words_0, words_1, words_2 = words[2:], words[1:-1], words[:-2]
    surp_0, surp_1, surp_2 = word_surprisals[2:], word_surprisals[1:-1], word_surprisals[:-2]

    df_story = pd.DataFrame.from_dict(
        {
            "word_0": words_0, "surp_0": surp_0,
            "word_1": words_1, "surp_1": surp_1,
            "word_2": words_2, "surp_2": surp_2
        }
    )

    words_pre = df[df["item"] == sentence_id+1].word.tolist()
    words_post = df_story.word_0.tolist()

    # make sure that the number of words is identical to that in the original data
    assert len(words_pre)-2 == len(words_post), f"Story {item}: {len(words_pre)-2}!={len(words_post)}"

    story_surprisals[sentence_id+1] = df_story

Next, we add another baseline predictor: The length of the 3 preceding words, in characters:

In [ ]:
# add position in sentence & sentence id as predictor
for item, df_story in story_surprisals.items():

    df_story["zone"] = range(1, len(df_story) +1)
    df_story["word_0_len"] = [len(word) for word in df_story.word_0]
    df_story["word_1_len"] = [len(word) for word in df_story.word_1]
    df_story["word_2_len"] = [len(word) for word in df_story.word_2]
    # needed for joining
    df_story["item"] = [item for _ in range(len(df_story))]

df_surp = pd.concat(story_surprisals.values())

Merging the surprisal/word length predictors into a single dataframe that will serve as the input to the LMER model:

In [ ]:
# Merge predictors into the new data frame
df_lmer = pd.merge(df_surp, df_rts, on=["item", "zone"], how="inner")
print(df_lmer.head(1))

We follow the common steps in the reading time prediction literature to 1) log-transform word frequencies and 2) center the raw reading times around zero.

In [ ]:
import numpy as np
df_lmer["word_0_freq"] = np.log(df_lmer.nItem)

In [ ]:
df_lmer["RT"] = (df_lmer["RT"] - np.mean(df_lmer["RT"]))/np.std(df_lmer["RT"])

We are now ready to fit a baseline LME model, that is, a regression model with the baseline predictors of world length and frequency with random intercepts for story (`item`) and word (`word_0`). The baseline model doesn't include predictors of surprisal. The quality of fit of our model to the reading time data is captured in the so-called log-likelihood of the model. 

A lower log-likelihood (compared to the baseline model) means that adding a predictor (surprisal in our case) *increases* the quality fit of the model. In the case of surprisal, this means that having it as one of the predictors yields more accurate predictions than not having it.



In [ ]:
from pymer4.models import Lmer

In [ ]:
model = Lmer(
    "RT ~ word_0_len \
        + word_1_len \
        + word_2_len \
        + word_0_freq \
        + (1|item) \
        + (1|word_0)",
    data=df_lmer
)

model.fit()

baseline_model_loglike = model.logLike

Another metric to evaluate the fit of our model is Mean Squared Error (MSE), which is - you guessed it -  the mean squared deviation of the predicted reading time $\hat{y}$ from the true reading time $y$:

$MSE = \frac{1}{N} \cdot \sum_i^{N} (y_i - \hat{y}_i)^2$

With $N$ in our case being the number of observed reading times. This is **not** equal to the number of surprisal values we obtained from GPT-2, since each word was read by more than one subject!

Define a MSE function (you can also use scikit-learn for that) and apply it to the true reading times and our predictions.

Note that MSE measures the amount of error in statistical models, specifically how close a regression line is to a set of data points. We want to minimize the error when fitting a model to data.

In [ ]:
def mse(model_fits, true_RTs) -> float:
    """
    Computes the mean squared error (MSE) between model fits and true reading times.

    Parameters:
    - model_fits: Predicted reading times from the model.
    - true_RTs: True reading times.

    Returns:
    - mse: Mean squared error between model fits and true reading times.
    """
    pass

In [ ]:
baseline_model_mse = mse(model.fits, df_lmer["RT"])

In [ ]:
print(f"Baseline model log-likelihood: {baseline_model_loglike}")
print(f"Baseline model MSE: {baseline_model_mse}")

Now that we have fit a baseline model, we are ready to add predictors of surprisal. Recall the meaning of `surp_0`, `surp_1`, and `surp_2`. We can use the measures of log-likelihood and MSE to assess the model's fit.

In [ ]:
%%capture
model = Lmer(
    "RT ~ surp_0 \
        + surp_1 \
        + surp_2 \
        + word_0_len \
        + word_1_len \
        + word_2_len \
        + word_0_freq \
        + (1|item) \
        + (1|word_0)",
    data=df_lmer
)

model.fit()

surp_model_mse = mse(model.fits, df_lmer["RT"])
surp_model_loglike = model.logLike

In [ ]:
print(f"Surprisal model log-likelihood: {surp_model_loglike}")
print(f"Surprisal model MSE: {surp_model_mse}")

Now, calculate the delta between the baseline model log-likelihood and the surprisal model log-likelihood.

In [ ]:
delta_loglike = surp_model_loglike - baseline_model_loglike
print(f"Delta log-likelihood: {delta_loglike}")

Note down this value and repeat the analysis for the surprisal values obtained from `gpt2-medium` and `gpt2-large`. 

1. Which model size yields the best fit? Why?

2. Do you find the same result for MSE?

### 4.  Calculate surprisal from LM outputs

This section explains how you can calculate surprisal from a LM without using the LM Toolkit (although the LM toolkit performs the same computations under the hood) with Hugging Face Transformers.

The Hugging Face transformers GPT2 model consists of three models: 1) The tokenizer, 2) The transformer model and 3) The language model 'head'.
The 'head' takes the output vector of the transformer model and obtains a vector of the size of the input vocabulary.

This vector is called 'logits'. As you may now, in Mathematics a 'logit' is a function that maps probabilities into the set of real numbers, i.e. $[0,1] \rightarrow \mathbb{R}$.
In the context of neural networks, 'logits' is used as a label for output $o$ with length $K = |o|$ of the model before the application of a softmax function, which is implied to "reverse" the log transformation of the probabilities.

$\text{softmax}(o)_i = \frac{e^{o_i}}{\sum_{j=1}^{K}e^{o_j}}$

Loading GPT2 with its language modelling head and tokenizer is quite simple. The code below might run for some time, since the model is downloaded from Hugging Face.

In [ ]:
import torch

Before we do that we should check if PyTorch is available and whether we're on a GPU or not:

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.to(device)
model.eval()

Since the GPT2 tokenizer doesn't have a pad token, we use the eos token instead.

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id

We need to obtain the token ids to index the tensor holding the log probabilities!
GPT-2 does not have a start of sequence symbol. We can emulate this by appending the end of sequence symbol to the tokenized sequences (this is what `prepend token` in the LM Toolkit web interface does). This is however not a true start of sequence symbol, and adding one would require retraining the embedding space. 

GPT-2 can take inputs of up to a size of 1024 tokens. This means that we have to break down each story into chunks of size $1024$. We do this by first tokenizing each story (resulting in subword sequences $> 1024$ tokens), and then rearranging it to fit into the model's context window:

In [ ]:
context_size = 1024

inputs = {}

for item in df["item"].unique():
    df_s = df[df.item==item]
    text = df_s.word.tolist()
    tokenized = tokenizer(" ".join(text))
    # split into chunks of size context_size - 1 & prepend eos token
    batch_reshaped = {
        k: [[tokenizer.eos_token_id if k == "input_ids" else 1] + v[i:i+context_size-1] for i in range(0,2*(context_size-1),context_size-1)] for k, v in tokenized.items()
    }

    # pad last sequence
    to_pad = context_size - len(batch_reshaped["input_ids"][-1])
    batch_reshaped["input_ids"][-1] = batch_reshaped["input_ids"][-1] + [tokenizer.pad_token_id for _ in range(to_pad)]
    batch_reshaped["attention_mask"][-1] = batch_reshaped["attention_mask"][-1] + [0 for _ in range(to_pad)]
    # copy labels from input ids
    batch_reshaped["labels"] = [input_id if input_id != tokenizer.pad_token_id else -100 for input_id in batch_reshaped["input_ids"]]
    inputs[item] = batch_reshaped

The logits attribute of the ```outputs``` object contains the output of the language modelling head. Let's define a function that applies softmax to the logits and convert the probabilities back to logarithmic scale:

In [ ]:
import torch.nn.functional as F

def get_logprobs(logits: torch.Tensor) -> torch.Tensor:
    """ Calculates perplexity from a list of log probabilities

    Parameters
    ----------
    logits : Tensor of shape (sequence_length, vocab_size)
        The tensor holding the language model outputs

    Returns
    -------
    Tensor
        The log-probabilities
    """

    # TODO: apply softmax to logits, take the negative log with base 2 (to obtain surprisal)


    raise NotImplementedError

The following code block runs GPT2 on the batches we constructed earlier, applies the above function to the outputs of the LM head (the logits) and retrieves the token surprisals by indexing the tensor that holds the transformed logits.

In [ ]:
from tqdm import tqdm

stories_scored = {}

for item, batch in tqdm(inputs.items()):

    batch = {k: torch.LongTensor(v).to(device) for k, v in batch.items()}
    
    with torch.no_grad(): # disable gradient computation
        outputs = model(**batch)
    
    token_surprisal = get_logprobs(outputs.logits)

    # we ignore the last surprisal value of each context window
    token_surprisal = token_surprisal[:, :-1].numpy().reshape(2*(context_size-1), tokenizer.vocab_size)

    # we ignore the first label of each context window
    output_ids = batch["labels"][:, 1:].numpy().squeeze().reshape(2*(context_size-1))

    # index for first dimension of surprisals
    index = torch.arange(0, output_ids.shape[0])
    token_surprisal = token_surprisal[index, output_ids]
    tokens = tokenizer.convert_ids_to_tokens(output_ids)

    assert len(tokens) == len(token_surprisal)

    stories_scored.update({item: (tokens, token_surprisal)})

Word-level surprisal can be calculated similarly to 2., with minor adjustments to the `get_word_surprisal` function. `token_surprisal` and `output_ids` still contain padding tokens and their surprisal values respectively. Adapt `get_word_surprisal` to ignore padding tokens and surprisal values:

In [ ]:
def get_word_surprisal(tokens: List[str], token_surprisal: List[float], pad_token: str) -> Tuple[List[str], List[float]]:
    """ Calculates per-word surprisal from a list of subword tokens and subword surprisal values
        by adding up the surprisal values of the subword tokens. Ignores padding tokens and their surprisal values.
    Parameters
    ----------
    tokens : List[str]
        The list of tokens. Each word-initial token is expected to start with the character 'Ġ',
        which is added by the GPT2 tokenizer.
    token_surprisal : List[float]
        The list of surprisal values corresponding to the subword tokens.

    Returns
    -------
    Tuple[List[str], List[float]]
        The reassembled words and their surprisal values.
    """
    
    word_surprisal = []
    words = []

    # TODO: Adapt your `get_word_surprisal` function from earlier to ignore padding tokens and surprisal values

    return words, word_surprisal

In order to measure how "surprised" GPT2 was by each of the stories (i.e. how well it could predict each word given the previous words), we calculate its so-called *perplexity* or the *branching factor* of the model. Perplexity is closely related to surprisal, and can be calculated as the exponential of the average surprisal on a sequence $w$:
$$
PP(w) = 2^{\frac{\sum_i^{|w|} -\log_2(w_i|w_{<i})}{|w|}}
$$

To evaluate the performance of GPT-2 on the corpus, complete the body of the perplexity function below and run it on the word surprisal values from the previous step:

In [ ]:
import numpy as np

def perplexity(surprisal: np.array) -> float:
    """ Calculates perplexity from a list of log probabilities

    Parameters
    ----------
    logprobs : List[float]
        The log probabilities

    Returns
    -------
    float
        The perplexity
    """
    raise NotImplementedError # TODO: return the calculated perplexity and remove this error after

In [ ]:
for item, (tokens, token_surprisals) in stories_scored.items():
    # step 1: get word surprisal
    words, word_surprisals = get_word_surprisal(tokens, token_surprisals, pad_token=tokenizer.pad_token)
    # step 2: calculate perplexity
    pp = perplexity(word_surprisals)
    print(item, np.round(pp,4))

If you are feeling very adventurous you can fit baseline and surprisal LMER models for each story separately, and then plot perplexity vs. the log-likelihood delta.

## References
[1] Oh, Byung-Doh, and William Schuler. “Why Does Surprisal From Larger Transformer-Based Language Models Provide a Poorer Fit to Human Reading Times?,” 2022. https://doi.org/10.48550/ARXIV.2212.12131.

[2] Futrell, Richard, Edward Gibson, Harry J. Tily, Idan Blank, Anastasia Vishnevetsky, Steven T. Piantadosi, and Evelina Fedorenko. “The Natural Stories Corpus: A Reading-Time Corpus of English Texts Containing Rare Syntactic Constructions.” Language Resources and Evaluation 55, no. 1 (2021): 63–77.
